# Clasificación de Señales de Tráfico con GTSRB

Este proyecto tiene como objetivo construir un modelo de **red neuronal convolucional (CNN)** para clasificar imágenes de señales de tráfico utilizando el dataset **GTSRB (German Traffic Sign Recognition Benchmark)**.

https://benchmark.ini.rub.de/gtsrb_dataset.html 

A continuación se documentan los pasos para cargar, preprocesar, entrenar y evaluar el modelo utilizando Python y TensorFlow.

---
## **Estructura de Directorios**

Asegúrate de que las imágenes y el código estén organizados de la siguiente manera:

C:/DB_Covid19Arg/csv_archivos_limpios/Amazon_test/ ├── GTSRB/ │ ├── Final_Training/ │ │ └── Images/ │ └── Final_Test/ │ └── Images/ └── train_model.py └── test_model.py


- **Final_Training**: Contiene las imágenes de entrenamiento organizadas en subdirectorios (uno por cada clase).
- **Final_Test**: Contiene las imágenes de prueba.




## Carga y Preprocesamiento de Datos

En esta sección, cargamos las imágenes de entrenamiento desde el directorio `Final_Training/Images/` y las preprocesamos. Las imágenes se redimensionan a 32x32 píxeles y se normalizan dividiendo los valores de los píxeles por 255.


In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

# Ruta a las imágenes de entrenamiento
train_images_path = 'C:/DB_Covid19Arg/csv_archivos_limpios/Amazon_test/GTSRB/Final_Training/Images/'

# Función para cargar las imágenes y sus etiquetas
def load_data(images_path):
    images = []
    labels = []
    for root, dirs, files in os.walk(images_path):
        for file in files:
            if file.endswith(".ppm"):  # Formato de imagen .ppm
                image = cv2.imread(os.path.join(root, file))
                image = cv2.resize(image, (32, 32))  # Redimensionar a 32x32 píxeles
                label = int(os.path.basename(root))  # El nombre del directorio es la etiqueta
                images.append(image)
                labels.append(label)
    return np.array(images), np.array(labels)

# Cargar imágenes y etiquetas
X, y = load_data(train_images_path)

# Normalizar los valores de las imágenes
X = X / 255.0

# Dividir los datos en entrenamiento y validación (80% entrenamiento y 20% validación)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Datos de entrenamiento: {X_train.shape}, Datos de validación: {X_val.shape}")


## Aumentación de Datos

Para evitar el sobreajuste durante el entrenamiento, aplicamos aumentación de datos usando `ImageDataGenerator`. Esto permite que el modelo vea variaciones de las imágenes durante el entrenamiento, como rotaciones, zoom y desplazamientos.


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Generador de imágenes con aumentación para evitar sobreajuste
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Ajustar el generador a los datos de entrenamiento
train_datagen.fit(X_train)


## Construcción del Modelo CNN

Definimos un modelo de red neuronal convolucional (CNN) usando `Sequential` de Keras. Este modelo tiene dos capas convolucionales seguidas de capas densas completamente conectadas. 


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Definir el modelo CNN
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(43, activation='softmax')  # 43 clases
])

# Compilar el modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


## Entrenamiento del Modelo

Entrenamos el modelo utilizando los datos de entrenamiento. Utilizamos un callback de `EarlyStopping` para detener el entrenamiento si la precisión de validación deja de mejorar.


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Early stopping para evitar sobreajuste
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Entrenar el modelo
history = model.fit(
    train_datagen.flow(X_train, y_train, batch_size=32),
    validation_data=(X_val, y_val),
    epochs=20,
    callbacks=[early_stopping]
)

# Guardar el modelo entrenado
model.save('traffic_sign_classifier.h5')

# Evaluar el modelo en el conjunto de validación
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")


from tensorflow.keras.callbacks import EarlyStopping

# Early stopping para evitar sobreajuste
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Entrenar el modelo
history = model.fit(
    train_datagen.flow(X_train, y_train, batch_size=32),
    validation_data=(X_val, y_val),
    epochs=20,
    callbacks=[early_stopping]
)

# Guardar el modelo entrenado
model.save('traffic_sign_classifier.h5')

# Evaluar el modelo en el conjunto de validación
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")


In [ ]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report, accuracy_score

# Ruta a las imágenes de prueba
test_images_path = 'C:/DB_Covid19Arg/csv_archivos_limpios/Amazon_test/GTSRB/Final_Test/Images/'

# Función para cargar las imágenes de prueba
def load_test_data(images_path):
    images = []
    filenames = []
    for root, dirs, files in os.walk(images_path):
        for file in files:
            if file.endswith(".ppm"):  # Formato de imagen .ppm
                image = cv2.imread(os.path.join(root, file))
                image = cv2.resize(image, (32, 32))  # Redimensionar a 32x32 píxeles
                images.append(image)
                filenames.append(file)
    return np.array(images), filenames

# Cargar imágenes de prueba
X_test, filenames = load_test_data(test_images_path)

# Normalizar las imágenes de prueba
X_test = X_test / 255.0

# Cargar el modelo entrenado
model = load_model('traffic_sign_classifier.h5')

# Realizar las predicciones en el conjunto de test
predictions = model.predict(X_test)
predicted_classes = np.argmax(predictions, axis=1)

# Imprimir predicciones
for filename, prediction in zip(filenames, predicted_classes):
    print(f"Imagen: {filename} - Clase predicha: {prediction}")
